In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lib.function import RobustAgent

## Random Parameter

In [ ]:
alpha=0
kappa_buy=15
kappa_sell=15
lambda_buy=2
lambda_sell=2
sigma=0.05
q_upper=3
q_lower=-3
Time=10
dt=0.01
theta=0.01
phi=0
phi_alpha=20
phi_lambda=10
phi_kappa=1

agent = RobustAgent(
    alpha=alpha,
    kappa_buy=kappa_buy,
    kappa_sell=kappa_sell,
    lambda_buy=lambda_buy,
    lambda_sell=lambda_sell,
    sigma=sigma,
    q_upper=q_upper,
    q_lower=q_lower,
    Time=Time,
    dt=dt,
    theta=theta,
    phi=phi,
    phi_alpha=phi_alpha,
    phi_kappa=phi_kappa,
    phi_lambda=phi_lambda,
)

fig, ax = plt.subplots(1, 2, figsize=(14, 5)) 

for q in range(q_upper - q_lower):
    ax[0].plot(agent.time_grid, agent.ops_uncertainty_num[0][:, q], color="blue")
    ax[0].plot(agent.time_grid, agent.ops_nouncertainty[0][:, q], '--', color="blue")
    # ax[0].plot(agent.time_grid, agent.ops_uncetainty_num[0][:, q], '--', color="red")
# plt.plot(agent.time_grid, agent.ops_uncertainty[0])
# plt.plot(agent.time_grid, agent.optimal_strategy_no_uncertainty[0], '--')
ax[0].set_xlabel("Time (secs)")
ax[0].set_ylabel("Sell Depth ($\delta^+$)")


for q in range(3):
    # q = 0 , 1 ,2 
    # index in sell is q_upper - q, index in buy is q_upper - 1 -q
    ax[1].plot(agent.time_grid, agent.ops_uncertainty_num[0][:, q_upper-q] + agent.ops_uncertainty_num[1][:, q_upper-1-q], color="blue")
    ax[1].plot(agent.time_grid, agent.ops_nouncertainty[0][:, q_upper-q] + agent.ops_nouncertainty[1][:, q_upper-1-q], '--',color="blue")
    # ax[1].plot(agent.time_grid, agent.ops_uncetainty_num[0][:, q_upper-q] + agent.ops_uncetainty_num[1][:, q_upper-1-q], '--',color="red")

ax[1].set_xlabel("Time (secs)")
ax[1].set_ylabel("Total Depth ($\delta^+ + \delta^-$)")
# ax[1].set_ylim(0.13, 0.16)

# fig.suptitle("")
plt.show()

## Compare the performance between robust control and plain control

In [ ]:
# Set the agent's strategy parameters 
alpha=0
kappa_buy=15
kappa_sell=15
lambda_buy=0.5
lambda_sell=0.5
sigma=0.1
q_upper=15
q_lower=-15
Time=10
dt=0.01
theta=0.01
phi=0
phi_alpha=20
phi_lambda=10
phi_kappa=1


In [ ]:
MC_samples = 20

MC_reward_robust = []
MC_reward_plain = []
# Set the environment parameters (add midprice drift/ different MO coming rate & filled rate)
env_alpha = 0.05
env_kappa_buy = 5
env_kappa_sell = 5
env_lambda_buy = 2
env_lambda_sell = 2

for _ in range(MC_samples):
    reward_with_uncertainty = []
    reward_without_uncertainty = []

    for _ in range(50):
        agent_with_uncertainty = RobustAgent(
            alpha=alpha,
            kappa_buy=kappa_buy,
            kappa_sell=kappa_sell,
            lambda_buy=lambda_buy,
            lambda_sell=lambda_sell,
            sigma=sigma,
            q_upper=q_upper,
            q_lower=q_lower,
            Time=Time,
            dt=dt,
            theta=theta,
            phi=phi,
            phi_alpha=phi_alpha,
            phi_kappa=phi_kappa,
            phi_lambda=phi_lambda,
        )

        agent_without_uncertainty = RobustAgent(
            alpha=alpha,
            kappa_buy=kappa_buy,
            kappa_sell=kappa_sell,
            lambda_buy=lambda_buy,
            lambda_sell=lambda_sell,
            sigma=sigma,
            q_upper=q_upper,
            q_lower=q_lower,
            Time=Time,
            dt=dt,
            theta=theta,
            phi=phi,
            phi_alpha=phi_alpha,
            phi_kappa=phi_kappa,
            phi_lambda=phi_lambda,
        )

        agent_with_uncertainty.run(control_uncertainty=True, env_alpha=env_alpha, env_lambda_buy=env_lambda_buy, env_lambda_sell=env_lambda_sell, env_kappa_buy=env_kappa_buy, env_kappa_sell=env_kappa_sell)
        agent_without_uncertainty.run(control_uncertainty=False, env_alpha=env_alpha, env_lambda_buy=env_lambda_buy, env_lambda_sell=env_lambda_sell, env_kappa_buy=env_kappa_buy, env_kappa_sell=env_kappa_sell)

        reward_with_uncertainty.append(agent_with_uncertainty.objective)
        reward_without_uncertainty.append(agent_without_uncertainty.objective)

    reward_with_uncertainty = np.array(reward_with_uncertainty)
    reward_without_uncertainty = np.array(reward_without_uncertainty)

    MC_reward_robust.append(np.mean(reward_with_uncertainty, axis=0))
    MC_reward_plain.append(np.mean(reward_without_uncertainty, axis=0))

MC_reward_robust = np.array(MC_reward_robust)
MC_reward_plain = np.array(MC_reward_plain)



In [ ]:
plt.plot(agent_with_uncertainty.time_grid, np.mean(reward_with_uncertainty, axis=0), label="Robust Strategy")
plt.plot(agent_without_uncertainty.time_grid, np.mean(reward_without_uncertainty, axis=0), '--',label="Plain Strategy")

plt.xlabel("Time")
plt.ylabel("Reward")
plt.legend()
plt.show()

In [ ]:
# Calculate mean and 95% CI
mean_robust = np.mean(MC_reward_robust, axis=0)
mean_plain = np.mean(MC_reward_plain, axis=0)

ci_robust = 1.96 * np.std(MC_reward_robust, axis=0) / np.sqrt(MC_samples)
ci_plain = 1.96 * np.std(MC_reward_plain, axis=0) / np.sqrt(MC_samples)

# Create x-axis values
ts = agent_with_uncertainty.time_grid

# Plot mean and CI
plt.plot(ts, mean_robust, label='Robust', color='blue')
plt.fill_between(ts, mean_robust - ci_robust, mean_robust + ci_robust, color='blue', alpha=0.2, label='95% CI Robust')

# Plot mean and CI
plt.plot(ts, mean_plain, '--', color='red', label='Plain')
plt.fill_between(ts, mean_plain - ci_plain, mean_plain + ci_plain, color='red', alpha=0.05, label='95% CI Plain')

plt.xlabel('Time')
plt.ylabel('Reward')
plt.title('Mean Reward with 95% Confidence Interval')
plt.legend()
plt.show()